In [1]:
import os
HOME = os.environ["HOME"]
os.environ["CARDIAC_GWAS_REPO"] = CARDIAC_GWAS_REPO = f"{HOME}/01_repos/CardiacGWAS"
os.environ["CARDIAC_COMA_REPO"] = CARDIAC_COMA_REPO = f"{HOME}/01_repos/CardiacCOMA/"
os.environ["GWAS_REPO"] = GWAS_REPO = f"{HOME}/01_repos/GWAS_pipeline/"

MLRUNS_DIR = f"{CARDIAC_COMA_REPO}/mlruns"
#os.chdir(CARDIAC_COMA_REPO)

In [2]:
import mlflow
from mlflow.tracking import MlflowClient

import os, sys

import torch
import torch.nn.functional as F

from CardiacCOMA.config.cli_args import overwrite_config_items
from CardiacCOMA.config.load_config import load_yaml_config, to_dict
from CardiacCOMA.utils.helpers import get_datamodule, get_lightning_module
from CardiacCOMA.utils.mlflow_helpers import get_model_pretrained_weights
from CardiacCOMA.utils.CardioMesh.CardiacMesh import transform_mesh

import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import Image, display, Markdown, clear_output

import pandas as pd
import shlex
from subprocess import check_output

import pickle as pkl
import pytorch_lightning as pl

from argparse import Namespace
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd
from IPython import embed
sys.path.insert(0, '..')

from copy import deepcopy
from pprint import pprint

from typing import List
from tqdm import tqdm

import pandas as pd

import pyvista as pv
from ipywidgets import interact, interactive, fixed, interact_manual

from auxiliary import load_data
from auxiliary import get_model_pretrained_weights

In [3]:
good_runs_df = pd.read_csv(f"{CARDIAC_GWAS_REPO}/results/good_runs.csv")
run_ids = good_runs_df.run_id.to_list()

In [4]:
def load_data(b):
    global meshes, procrustes_transforms
    print("Loading mesh data...")
    meshes = pkl.load(open(f"{CARDIAC_COMA_REPO}/data/cardio/LV_meshes_at_ED_35k.pkl", "rb"))
    print("Mesh data loaded successfully.")
    
    print("Loading Procrustes transforms...")
    procrustes_transforms = pkl.load(open(f"{CARDIAC_COMA_REPO}/data/cardio/procrustes_transforms_35k.pkl", "rb"))
    print("Procrustes transform loaded successfully.")
    
button = widgets.Button(
    description='Load data',
    disabled=False,
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)

load_data(1)
#button.on_click(load_data)
#button    

Loading mesh data...
Mesh data loaded successfully.
Loading Procrustes transforms...
Procrustes transform loaded successfully.


In [5]:
ids = [ str(id) for id in meshes ] 

In [6]:
import random
pv.set_plot_theme("document")

faces, _ = pkl.load(open(f"{CARDIAC_COMA_REPO}/data/cardio/faces_and_downsampling_mtx_frac_0.1_LV.pkl", "rb")).values()
faces = np.c_[np.ones(faces.shape[0]) * 3, faces].astype(int)

color_palette = list(pv.colors.color_names.values())
random.shuffle(color_palette)

# Select genetic locus

In [7]:
gwas_harmonized_pattern = "data/other_gwas/preprocessed_files/{prefix}__{phenotype}.tsv"
gwas_selected_snps_pattern = "data/other_gwas/preprocessed_files/{prefix}__{phenotype}__selected_snps.tsv"
COMA_GWAS_SUMMARY = "results/gwas_loci_summary_across_runs.csv"
LOGP_PATH = "results/log10p_for_selected_snps_across_gwas.csv"

In [8]:
gwas_loci_summary_across_runs_df = pd.read_csv(COMA_GWAS_SUMMARY)

# get index of best locus/variable
idx = gwas_loci_summary_across_runs_df.groupby(["region"])["P"].transform(min) == gwas_loci_summary_across_runs_df["P"]

best_association_per_region = gwas_loci_summary_across_runs_df[idx].sort_values("region")
best_snps = set(best_association_per_region.SNP)

regions = { 
    f"{assoc[1].region} ({assoc[1].P:.1e})": assoc[1].region 
    for assoc in best_association_per_region.sort_values("P").iterrows() 
} 

del regions["chr6_79 (4.5e-20)"]

_best_association_per_region = best_association_per_region.set_index("region")
# assoc = _best_association_per_region.loc[region]
#run_id, z_variable = assoc.run, assoc.pheno[-4:]

In [9]:
_best_association_per_region.head(5)

,run,pheno,CHR,SNP,BP,AF,a_0,a_1,BETA,SE,T,P,locus_name
region,,,,,,,,,,,,,
chr10_20,e6490cf8389f4e7da52fef013562770f,1_e6490_z012,10.0,rs2797593,28655648.0,0.358680,A,C,0.049402,0.008336,5.9265,3.126079e-09,Unnamed
chr10_69,8b630b3a67f94ec2853645fe0314a424,1_8b630_z003,10.0,rs189569984,112544125.0,0.008998,C,T,-0.246570,0.041965,-5.8757,4.253045e-09,RBM20
chr10_74,b0ca240bf966455888005eee1e1fed99,1_b0ca2_z004,10.0,rs375034445,121424815.0,0.214880,A,AT,-0.056170,0.009752,-5.7600,8.489850e-09,Unnamed
chr11_2,a7d3200ce0fb43d586d53db5afff50d9,1_a7d32_z005,11.0,rs569550,1887068.0,0.386020,T,G,-0.057778,0.008179,-7.0643,1.648162e-12,LSP1*
chr11_32,cc438c4acf664d5ab05ed79686968a06,1_cc438_z003,11.0,rs371701829,57864175.0,0.321240,CT,C,0.053399,0.008821,6.0537,1.431858e-09,Unnamed


In [10]:
N_STEPS = 100
STEP = 1 / N_STEPS
quantiles = np.arange(N_STEPS+1)/N_STEPS

quantile_ranges = [(quantiles[i], quantiles[i+1]) for i, q in enumerate(quantiles[:-1])]

In [11]:
from PIL import Image
import imageio

def merge_pngs(pngs, output_png, how):
    # https://www.tutorialspoint.com/python_pillow/Python_pillow_merging_images.htm
    
    # Read images    
    images = [Image.open(png) for png in pngs]    
    
    x_sizes = [image.size[0] for image in images]
    y_sizes = [image.size[1] for image in images]
    
    if how == "vertically":      
      y_size = sum(y_sizes)  
      x_size = images[0].size[0]      
      y_sizes.insert(0, 0)      
      y_positions = np.cumsum(y_sizes[:-1])    
      positions = [(0, y_position) for y_position in y_positions]
    
    elif how == "horizontally":
      x_size = sum(x_sizes)      
      y_size = images[0].size[1]      
      x_sizes.insert(0, 0)      
      x_positions = np.cumsum(x_sizes[:-1])    
      positions = [(x_position, 0) for x_position in x_positions]
    
    
    new_image = Image.new(
        mode='RGB',
        size=(x_size, y_size),
        color=(250, 250, 250)
    )
    
    for i, image in enumerate(images):        
        new_image.paste(image, positions[i])
        
    new_image.save(output_png, "PNG")

In [12]:
import pickle as pkl

In [13]:
#aligned_meshes = { id:  transform_mesh(mesh,  **procrustes_transforms[id]) for id, mesh in meshes.items() }
#rmsd = {id: np.array([np.sqrt((p**2).sum()) for p in mesh]).mean() for id, mesh in aligned_meshes.items()}
#pkl.dump(aligned_meshes, open("lved_aligned_meshes.pkl", "wb"))
#pkl.dump(rmsd, open("lved_rmsd.pkl", "wb"))

In [14]:
aligned_meshes = pkl.load(open("lved_aligned_meshes.pkl", "rb"))
rmsd = pkl.load(open("lved_rmsd.pkl", "rb"))

In [15]:
# 
# scaled_meshes = []
# msd = []
#             
# for id in ids:
#     
#     mesh = transform_mesh(
#         meshes[id], 
#         **procrustes_transforms[id]
#     )
#         
#     # mean-squared deviation
#     msd.append(np.array([np.sqrt((p**2).sum()) for p in mesh]).mean())
#     scaled_meshes.append(mesh)
#     
# scaled_meshes = np.array(scaled_meshes)
# msd = np.array(msd)

In [16]:
from typing import Sequence, Dict

def get_ids_in_range(z: pd.Series, q0: float, q1: float):
    '''
      z: pd.Series 
      q0, q1: quantile bounds (q0 < q1, between 0 and 1)    
    '''
        
    z_bounds = z.quantile([q0, q1])    
    ids_in_range = z[(z_bounds[q0] < z) & (z < z_bounds[q1])].index
    
    # coerce id's to string and convert to list
    ids_in_range = [str(id) for id in ids_in_range]
    
    return ids_in_range


def avg_shape_in_q_range(
        meshes: Dict[str, np.array], 
        rmsd: Dict[str, float],
        quantile_range: tuple[float], 
        run_id: str, z: str, exp_id: str ="1"
    ):
    
    '''
    meshes: np.array with dimension N x 3.
    quantile_range: a tuple with two quantiles (floats between 0 and 1 representing).
    z: name of the z variable (*not* the values) 
    '''
    
    z_filepath = f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/latent_vector.csv"
    z = pd.read_csv(z_filepath).set_index("ID")[z]
    
    ids_in_range = get_ids_in_range(z, q0, q1)
    
    avg_mesh = np.array([meshes[id] / rmsd[id] for id in ids_in_range]).mean(0)
    rmsd_in_range = np.array([rmsd[id] for id in ids_in_range]).mean()
    avg_mesh = avg_mesh * rmsd_in_range
        
    return avg_mesh 


def plot_mesh(mesh, faces, ofilename):#, camera=(300, 0.0, 0.0):
    
    pv.set_plot_theme("document")
    pl = pv.Plotter(off_screen=True, notebook=False)
    
    pl.camera.position = (300, 0.0, 0.0)
    pl.camera.azimuth = 95
        
    mesh = pv.PolyData(mesh, faces)
    
    pl.add_mesh(
        mesh, show_edges=False, point_size=1.5, color=color_palette[0], opacity=0.5
    )
    
    pl.screenshot(ofilename);  


In [17]:
ids_in_range = list(meshes.keys())[0:100]
avg_shape = np.array([meshes[id] / rmsd[id] for id in ids_in_range]).mean(0)
rmsd_in_range = np.array([rmsd[id] for id in ids_in_range]).mean()
avg_shape *= rmsd_in_range
avg_shape

array([[315.33873711,  67.55932002, 286.03773234],
       [297.8330854 ,  59.7974544 , 202.14065344],
       [320.81589542,  49.9876801 , 281.62826188],
       ...,
       [259.13623771,  62.71776302, 263.8711187 ],
       [321.99926626,  67.93798576, 255.43474741],
       [295.96140628,  55.19356009, 203.28433615]])

In [18]:
exp_id = '1'

regions = [
   "chr17_27",
   "chr12_69",
   "chr6_78",
   "chr2_108",
   "chr11_2",
   "chr5_103",
   "chr1_124",
   "chr6_84",
   "chr2_69",
   "chr12_67"
]

q_ranges = [(0.00, 0.01), (0.095, 0.105), (0.45, 0.55), (0.895, 0.905), (0.99, 1.0)]

In [19]:
import warnings
warnings.filterwarnings('ignore')

In [20]:
VERBOSE = False

for run_id in sorted(os.listdir(f"{CARDIAC_COMA_REPO}/mlruns/1")):
    
    try:
        z_filepath = f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/latent_vector.csv"
        zs = pd.read_csv(z_filepath).set_index("ID").columns
    except:
        continue
        
    for z in zs:
        
        # print(f"{run_id[:5]}/{z}")
        
        #assoc = _best_association_per_region.loc[region]
        #run_id, z = assoc.run, assoc.pheno[-4:]
    
        z_effect_figs_dir = f"{CARDIAC_COMA_REPO}/mlruns/1/{run_id}/artifacts/z_effect_on_shape"
        os.makedirs(z_effect_figs_dir, exist_ok=True)
        
        filenames = { 
            (q0, q1): f"{z_effect_figs_dir}/{z}_{q0}-{q1}.png" for q0, q1 in q_ranges 
        }
    
        for (q0, q1), filename in filenames.items():
              
            if os.path.exists(filename):
                if VERBOSE: print(f"File {filename} already exists.")
                continue                        
            
            avg_mesh = avg_shape_in_q_range(                
                meshes=aligned_meshes,
                rmsd=rmsd,
                quantile_range=(q0, q1),             
                exp_id=exp_id,
                run_id=run_id,
                z=z,            
            )  
            
            if np.isnan(avg_mesh).all(): break
            
            # print(f"{run_id[:5]}/{z}")
            
            ofilename = filenames[(q0, q1)]
            plot_mesh(avg_mesh, faces, ofilename)                                    
           
        try:            
            filename = f"{z_effect_figs_dir}/{z}.png"
            if not os.path.exists(filename):
                merge_pngs(filenames.values(), output_png=filename, how="horizontally")
        except:
            pass
        
    print(f"{run_id[:5]}")

0285f
03e46
049c2
04a59
06413
06eef
09565
0ddcb
10001
10284
136de
17629
1ceac
1d8d4
1f840
21f83
23689
24b8a
25cd6
26568
28a33
28cdc
293f5
2965f
2c83b
2dce0
3240f
32c57
346c4
36693
368a7
37a9f
37afc
37b6f
37b74
383a4
394bf
3bb27
3ccf1
3e055
3ef97
40631
416a8
41923
44e53
45e3c
46cd6
470ab
4a637
4b73b
4b765
4db72
4eedb
4fb49
50df4
586b4
58e79
5ade7
5b82e
5bf27
5de82
5e2d1
5e718
61835
6277c
64bde
65535
67d13
6b492
6befa
6c5c1
6cc87
6e096
6ffc5
71631
71ab6
72116
73826
74b6e
77bed
77fd5
7842a
7963e
8025c
812a8
81c75
83e5d
84a1c
854f2
8614c
882dc
8b630
8d082
8d910
8ea1d
8f24c
92337
94afe
956de
9790e
97ea3
98525
9872e
9976e
9a4b9
9a56d
9a924
9c46c
9dd2c
9eaff
9ee36
a156b
a5b2d
a6bde
a7d32
a8c4d
aabda
ade69
aff30
b0420
b0ca2
b1542
b23e7
b4fc6
b526b
b6a7e
b6a9c
b6f4c
ba93d
bae5e
bc2d5
c1f26
c3b17
c7f74
cc438
d02d6
d2850
d38cd
d7d5e
d9579
da264
da341
da5d1
db8b7
e1928
e4283
e4c27
e599d
e6490
e670f
e75e7
e8a19
e93c4
ee30f
f0978
f29b0
f8a58
f9c17
fdc46
ff53b
ff7c5


In [21]:
for region in tqdm(regions):
  
  assoc = _best_association_per_region.loc[region]
  run_id, z = assoc.run, assoc.pheno[-4:]

  df = pd.read_csv(f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/latent_vector.csv").set_index("ID")

  filenames = []

  for q0, q1 in q_ranges:  
    
    filename = f"results/figs/{region}_{q0}-{q1}.png"
    print(filename)
    filenames.append(filename)  
    
    if os.path.exists(filename):
        print(f"File {filename} already exists.")
        continue
    
    z_bounds = df.quantile([q0, q1])[z]                
        
    ids = list(df[ 
        (z_bounds[q0] < df[z]) & (df[z] < z_bounds[q1])
    ][z].index) 
      
    scaled_meshes = []
    msd = []
    
    for id in ids:
        mesh = transform_mesh(
            meshes[str(id)], 
            **procrustes_transforms[str(id)]
        )
        
        # mean-squared deviation
        msd.append(np.array([np.sqrt((p**2).sum()) for p in mesh]).mean())
        scaled_meshes.append(mesh)
    
    scaled_meshes = np.array(scaled_meshes)
    msd = np.array(msd)
            
    avg_mesh = np.array([scaled_meshes[i] / msd[i] for i, _ in enumerate(scaled_meshes)]).mean(0)
    avg_mesh = avg_mesh * msd.mean()
                
    pv.set_plot_theme("document")
    pl = pv.Plotter(off_screen=True, notebook=False)
    
    pl.camera.position = (300, 0.0, 0.0)
    pl.camera.azimuth = 95
    
    mesh = pv.PolyData(avg_mesh, faces)
    pl.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[0], opacity=0.5)
    
    pl.screenshot(filename);
    
  
  merge_pngs(filenames, output_png=f"{region}.png", how="horizontally")

  0%|                                                                                                                                            | 0/10 [00:00<?, ?it/s]

results/figs/chr17_27_0.0-0.01.png
File results/figs/chr17_27_0.0-0.01.png already exists.
results/figs/chr17_27_0.095-0.105.png
File results/figs/chr17_27_0.095-0.105.png already exists.
results/figs/chr17_27_0.45-0.55.png
File results/figs/chr17_27_0.45-0.55.png already exists.
results/figs/chr17_27_0.895-0.905.png
File results/figs/chr17_27_0.895-0.905.png already exists.
results/figs/chr17_27_0.99-1.0.png
File results/figs/chr17_27_0.99-1.0.png already exists.


 10%|█████████████▏                                                                                                                      | 1/10 [00:00<00:02,  3.59it/s]

results/figs/chr12_69_0.0-0.01.png
File results/figs/chr12_69_0.0-0.01.png already exists.
results/figs/chr12_69_0.095-0.105.png
File results/figs/chr12_69_0.095-0.105.png already exists.
results/figs/chr12_69_0.45-0.55.png
File results/figs/chr12_69_0.45-0.55.png already exists.
results/figs/chr12_69_0.895-0.905.png
File results/figs/chr12_69_0.895-0.905.png already exists.
results/figs/chr12_69_0.99-1.0.png
File results/figs/chr12_69_0.99-1.0.png already exists.


 20%|██████████████████████████▍                                                                                                         | 2/10 [00:00<00:02,  3.75it/s]

...

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▌                          | 8/10 [00:57<00:06,  3.22s/it]

results/figs/chr2_69_0.0-0.01.png
File results/figs/chr2_69_0.0-0.01.png already exists.
results/figs/chr2_69_0.095-0.105.png
File results/figs/chr2_69_0.095-0.105.png already exists.
results/figs/chr2_69_0.45-0.55.png
File results/figs/chr2_69_0.45-0.55.png already exists.
results/figs/chr2_69_0.895-0.905.png
File results/figs/chr2_69_0.895-0.905.png already exists.
results/figs/chr2_69_0.99-1.0.png
File results/figs/chr2_69_0.99-1.0.png already exists.


 90%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊             | 9/10 [00:57<00:02,  2.30s/it]

results/figs/chr12_67_0.0-0.01.png
File results/figs/chr12_67_0.0-0.01.png already exists.
results/figs/chr12_67_0.095-0.105.png
File results/figs/chr12_67_0.095-0.105.png already exists.
results/figs/chr12_67_0.45-0.55.png
File results/figs/chr12_67_0.45-0.55.png already exists.
results/figs/chr12_67_0.895-0.905.png
File results/figs/chr12_67_0.895-0.905.png already exists.
results/figs/chr12_67_0.99-1.0.png
File results/figs/chr12_67_0.99-1.0.png already exists.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:58<00:00,  5.81s/it]


In [22]:
exp_id = '1'
# region = "chr17_27"
region = "chr12_69"
assoc = _best_association_per_region.loc[region]
run_id, z = assoc.run, assoc.pheno[-4:]
filename = "latent_vector.csv"
df = pd.read_csv(f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/{filename}")
df = df.set_index("ID")

for q0, q1 in quantile_ranges:
  
    z_bounds = df.quantile([q0, q1])[z]                
    
    ids = list(df[ 
        (z_bounds[q0] < df[z]) & (df[z] < z_bounds[q1])
    ][z].index) 
    
    scaled_meshes = []
    msd = []
    
    for id in ids:
        mesh = transform_mesh(
            meshes[str(id)], 
            **procrustes_transforms[str(id)]
        )
        
        # mean-squared deviation
        msd.append(np.array([np.sqrt((p**2).sum()) for p in mesh]).mean())
        scaled_meshes.append(mesh)
    
    scaled_meshes = np.array(scaled_meshes)
    
    msd = np.array(msd)            
    avg_mesh = np.array([scaled_meshes[i] / msd[i] for i, _ in enumerate(scaled_meshes)]).mean(0)
    avg_mesh = avg_mesh * msd.mean()
    
    pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
    mesh = pv.PolyData(avg_mesh, faces)
    pl.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[0], opacity=0.5)
    
    clear_output()
    print()
    pl.show(interactive=True, interactive_update=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [23]:
exp_id = '1'
STEP = 0.01

region_w = widgets.Select(options=regions, description="Locus: \n",)
display(region_w)
region = region_w.value

# _best_association_per_region = best_association_per_region.set_index("region")
assoc = _best_association_per_region.loc[region]
run_id, z = assoc.run, assoc.pheno[-4:]

quantile_inf_w=widgets.FloatSlider(min=0.00, max=0.99, step=STEP)
display(quantile_inf_w)    

button = widgets.Button(description="Plot mesh")
out = widgets.Output()

def plot_mesh(b):
    
    filename = "latent_vector.csv"
    df = pd.read_csv(f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/{filename}")
    df = df.set_index("ID")
    
    # z_w = widgets.Select(options=[f"z{str(i).zfill(3)}" for i in range(df.shape[1])])    
    
    with out:

        quantile_range = quantile_inf_w.value
        z_bounds = df.quantile([quantile_range, STEP+quantile_range])[z]                
        
        ids = list(df[ 
            (z_bounds[quantile_range] < df[z]) & (df[z] < z_bounds[quantile_range+STEP])
        ][z].index)    
                
        # COMPUTE AVERAGE MESH                        
        avg_mesh = np.array([
            transform_mesh(
                meshes[str(id)], 
                **procrustes_transforms[str(id)]
            ) for id in ids
        ]).mean(axis=0)

        clear_output()        
        
        pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
        mesh = pv.PolyData(avg_mesh, faces)
        pl.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[0], opacity=0.5)
        pl.show(interactive=True, interactive_update=True)
        
button.on_click(plot_mesh)
# displaying button and its output together
widgets.VBox([button,out])    

Select(description='Locus: \n', options=('chr17_27', 'chr12_69', 'chr6_78', 'chr2_108', 'chr11_2', 'chr5_103',…

FloatSlider(value=0.0, max=0.99, step=0.01)

___

# Select `run_id` / `z` variable

In [24]:
from PIL import Image

exp_id = '1'
STEP=0.01
run_id_w = widgets.Select(options=sorted(run_ids))
display(run_id_w)

filename = "latent_vector.csv"
df = pd.read_csv(f"{MLRUNS_DIR}/{exp_id}/{run_id}/artifacts/output/{filename}")
df = df.set_index("ID")

z_w = widgets.SelectionSlider(options=[f"z{str(i).zfill(3)}" for i in range(df.shape[1])])
display(z_w)

quantile_range_w=widgets.FloatSlider(min=0, max=0.99, step=STEP)
display(quantile_range_w)

button = widgets.Button(description="Plot mesh")

out = widgets.Output()


def plot_mesh(b):
    
    with out:

        z = z_w.value
        run_id = run_id_w.value
        quantile_range = quantile_range_w.value
        z_bounds = df.quantile([quantile_range_w.value, quantile_range_w.value+STEP])[z]                
        
        ids = list(df[ 
            (z_bounds[quantile_range_w.value] < df[z]) & (df[z] < z_bounds[quantile_range_w.value+STEP])
        ][z].index) 
        
        print(len(ids))
                
        clear_output()      
        print(z, run_id)
        manhattan_file = f"{CARDIAC_COMA_REPO}/mlruns/1/{run_id}/artifacts/GWAS_adj_10PCs/figures/GWAS__{z}__1_{run_id}__manhattan.png"
        qq_file = f"{CARDIAC_COMA_REPO}/mlruns/1/{run_id}/artifacts/GWAS_adj_10PCs/figures/GWAS__{z}__1_{run_id}__QQ-plot.png"
        
        # COMPUTE AVERAGE MESH                        
        avg_mesh = np.array([
            transform_mesh(
                meshes[str(id)], 
                **procrustes_transforms[str(id)]
            ) for id in ids
        ]).mean(axis=0)

                      
        pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)
        mesh = pv.PolyData(avg_mesh, faces)
        pl.add_mesh(mesh, show_edges=False, point_size=1.5, color=color_palette[0], opacity=0.5)
        pv.camera
        pl.show(interactive=True, interactive_update=True)
        
        display(Image.open(manhattan_file))
        # display(Image.open(qq_file))
        
button.on_click(plot_mesh)
# displaying button and its output together
widgets.VBox([button,out])        

Select(options=('0285fa2356fd454e88e3c30d6b63f163', '064136c02b284aeeb946de907b5ed100', '09565efcb04642ca8d6bb…

SelectionSlider(options=('z000', 'z001', 'z002', 'z003', 'z004', 'z005', 'z006', 'z007'), value='z000')

FloatSlider(value=0.0, max=0.99, step=0.01)

In [25]:
pl = pv.Plotter(notebook=True, off_screen=False, polygon_smoothing=False)